In [1]:
!pip3 install -qU tf2onnx onnxruntime

# TODO: update training-v2 by main after merge of training-v2
!pip3 install -q git+https://github.com/vasudevgupta7/gsoc-wav2vec2@training-v2

     |████████████████████████████████| 398 kB 5.0 MB/s 
     |████████████████████████████████| 4.5 MB 51.5 MB/s 
     |████████████████████████████████| 12.3 MB 176 kB/s 
     |████████████████████████████████| 1.8 MB 5.0 MB/s 
     |████████████████████████████████| 43 kB 1.7 MB/s 
     |████████████████████████████████| 50 kB 5.4 MB/s 
     |████████████████████████████████| 1.7 MB 27.5 MB/s 
     |████████████████████████████████| 133 kB 42.0 MB/s 
     |████████████████████████████████| 170 kB 46.6 MB/s 
     |████████████████████████████████| 188 kB 61.8 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 


In [2]:
import tensorflow as tf

In [4]:
from wav2vec2 import Wav2Vec2ForCTC

model_id = "vasudevgupta/gsoc-wav2vec2-base-960h"
model = Wav2Vec2ForCTC.from_pretrained(model_id)

Total number of loaded variables: 213


In [6]:
!wget https://www.openslr.org/resources/12/test-clean.tar.gz
!tar -xf test-clean.tar.gz

--2021-08-08 00:38:28--  https://www.openslr.org/resources/12/test-clean.tar.gz
Resolving www.openslr.org (www.openslr.org)... 46.101.158.64
Connecting to www.openslr.org (www.openslr.org)|46.101.158.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 346663984 (331M) [application/x-gzip]
Saving to: ‘test-clean.tar.gz’

test-clean.tar.gz   100%[===================>] 330.60M  27.1MB/s    in 14s     

2021-08-08 00:38:43 (24.2 MB/s) - ‘test-clean.tar.gz’ saved [346663984/346663984]



In [11]:
import soundfile as sf
# speech, _ = sf.read("../data/SA2.wav")
# speech = tf.expand_dims(tf.constant(speech, dtype=tf.float32), 0)
# speech.shape

AUDIO_MAXLEN = 246000

speech = tf.random.uniform(shape=(1, 246000))

In [13]:
import tf2onnx
ONNX_PATH = "onnx-wav2vec2.onnx"

input_signature = (tf.TensorSpec((None, AUDIO_MAXLEN), tf.float32, name="speech"),)
_ = tf2onnx.convert.from_keras(model, input_signature=input_signature, output_path=ONNX_PATH)

In [14]:
import onnxruntime as rt

sess = rt.InferenceSession(ONNX_PATH)
onnx_outputs = sess.run(None, {"speech": speech.numpy()})
onnx_outputs

[array([[[  4.952241 , -17.584637 , -17.482151 , ...,  -2.0811079,
           -2.8112628,  -6.3643446],
         [  4.944199 , -17.5805   , -17.47725  , ...,  -2.0816162,
           -2.8053281,  -6.373602 ],
         [  4.95032  , -17.57232  , -17.469223 , ...,  -2.0803037,
           -2.7945876,  -6.3585234],
         ...,
         [  4.93646  , -17.574364 , -17.47243  , ...,  -2.0959978,
           -2.8019748,  -6.3186812],
         [  4.9474607, -17.597885 , -17.494541 , ...,  -2.0905585,
           -2.8233068,  -6.342802 ],
         [  4.9448576, -17.573168 , -17.470783 , ...,  -2.0908818,
           -2.785866 ,  -6.3446217]]], dtype=float32)]

In [15]:
import time

start = time.time()
onnx_outputs = sess.run(None, {"speech": speech.numpy()})
print(onnx_outputs)
print("TIME TAKEN:", time.time() - start)

[array([[[  4.952241 , -17.584637 , -17.482151 , ...,  -2.0811079,
          -2.8112628,  -6.3643446],
        [  4.944199 , -17.5805   , -17.47725  , ...,  -2.0816162,
          -2.8053281,  -6.373602 ],
        [  4.95032  , -17.57232  , -17.469223 , ...,  -2.0803037,
          -2.7945876,  -6.3585234],
        ...,
        [  4.93646  , -17.574364 , -17.47243  , ...,  -2.0959978,
          -2.8019748,  -6.3186812],
        [  4.9474607, -17.597885 , -17.494541 , ...,  -2.0905585,
          -2.8233068,  -6.342802 ],
        [  4.9448576, -17.573168 , -17.470783 , ...,  -2.0908818,
          -2.785866 ,  -6.3446217]]], dtype=float32)]
TIME TAKEN: 4.563531160354614


In [16]:
start = time.time()
outputs = model(speech)
print(outputs)
print("TIME TAKEN:", time.time() - start)

tf.Tensor(
[[[  4.952262  -17.584675  -17.482185  ...  -2.0811026  -2.811307
    -6.364362 ]
  [  4.9442177 -17.580526  -17.47728   ...  -2.0816128  -2.805375
    -6.37362  ]
  [  4.9503393 -17.572357  -17.469252  ...  -2.0802975  -2.7946343
    -6.358538 ]
  ...
  [  4.9364786 -17.5744    -17.472464  ...  -2.0959911  -2.8020182
    -6.3186984]
  [  4.947481  -17.597918  -17.49458   ...  -2.0905519  -2.8233542
    -6.3428206]
  [  4.944875  -17.573202  -17.470823  ...  -2.0908775  -2.785912
    -6.3446407]]], shape=(1, 768, 32), dtype=float32)
TIME TAKEN: 5.378181457519531


In [17]:
@tf.function(jit_compile=True)
def forward(speech):
    return model(speech)

In [19]:
start = time.time()
outputs = forward(speech)
print(outputs)
print("TIME TAKEN:", time.time() - start)

tf.Tensor(
[[[  4.9522552 -17.584661  -17.48217   ...  -2.08113    -2.811292
    -6.364342 ]
  [  4.9442124 -17.580513  -17.477264  ...  -2.0816388  -2.8053577
    -6.373598 ]
  [  4.9503317 -17.57234   -17.469234  ...  -2.0803254  -2.7946198
    -6.3585196]
  ...
  [  4.936473  -17.574383  -17.472452  ...  -2.096019   -2.8020055
    -6.3186784]
  [  4.9474735 -17.597902  -17.494568  ...  -2.090579   -2.8233392
    -6.342799 ]
  [  4.9448686 -17.573189  -17.470806  ...  -2.0909042  -2.785896
    -6.3446193]]], shape=(1, 768, 32), dtype=float32)
TIME TAKEN: 12.117019414901733


In [ ]:
import numpy as np
np.allclose(onnx_outputs, outputs.numpy(), atol=1e-2)

True